# Scraping trail running races 2021

This notebook scraps the trail running races's data from https://itra.run/races

SyntaxError: invalid syntax (<ipython-input-1-3e81a140bc65>, line 1)

In [6]:
## Run in shell

# Virtual environment 
conda env list
conda create --name certTrail -k
conda activate certTrail

# Libraries needed
conda install selenium -k
conda install beautifulsoup4 -k
conda install lxml -k
pip install html5lib #easy_install html5lib
pip install whois #easy_install whois
pip install builtwith #easy_install builtwith
conda install -c conda-forge geckodriver -k

SyntaxError: invalid syntax (<ipython-input-6-d99c37cf79ed>, line 4)

In [71]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import logging
import concurrent.futures
import time
import re
import json
import pandas as pd
#import whois
import sys
from tqdm import tqdm

# What environment am I using?
print(sys.executable)

C:\Anaconda\python.exe


In [100]:
#Setup options for FireFox
from selenium.webdriver.firefox.options import Options as FirefoxOptions

options = FirefoxOptions()
options.binary_location = r'C:\Users\apelaez\AppData\Local\Mozilla Firefox\firefox.exe'
options.add_argument("--lang=en-US")

# Using selenium, open firefox window with the ITRA website
driver = webdriver.Firefox(options=options)
driver.get("https://itra.run/races")

# Getting current URL source code 
get_title = driver.title 
  
# Printing the title of this URL 
print(get_title) 
assert "- ITRA" in driver.title


Races - ITRA


In [107]:
# Click dropdown menu for language selection
#driver.find_element_by_xpath("/html/body/div[1]/div[1]/nav/div[4]").click()

#Switch to English
btnCurrentLanguage = driver.find_element_by_css_selector("div.nav-item.dropdown")
btnCurrentLanguage.click()

dpkStartdate = driver.find_element_by_css_selector("div.vdp-datepicker.dp1")
dpkStartdate.click()

spnMonthSelector = driver.find_element_by_css_selector("span.day__month_btn.up")
spnMonthSelector.click()

spnYearSelector = driver.find_element_by_css_selector("span.month__year_btn.up")
spnYearSelector.click()

divYears = driver.find_elements_by_css_selector("span.cell.year")
print(len(divYears))

20


In [51]:
# Click on PERIOD time selection "START"
driver.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[1]/div[2]/div/div[2]/div/div[1]/div/div[1]/div[1]/input").click()

# Click on month selection
driver.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[1]/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/header/span[2]").click()

# Click on January
driver.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[1]/div[2]/div/div[2]/div/div[1]/div/div[1]/div[3]/span[1]").click()

# Click on 1st
driver.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[1]/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/div/span[13]").click()


In [18]:
# Click on PERIOD time selection "END"
driver.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[1]/div[2]/div/div[2]/div/div[1]/div/div[4]/div[1]/input").click()

# Click on month selection
driver.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[1]/div[2]/div/div[2]/div/div[1]/div/div[4]/div[2]/header/span[2]").click()

# Click on year
driver.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[1]/div[2]/div/div[2]/div/div[1]/div/div[4]/div[3]/header/span[2]").click()

# Click on 2021
driver.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[1]/div[2]/div/div[2]/div/div[1]/div/div[4]/div[4]/span[2]").click()

# Click on December
driver.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[1]/div[2]/div/div[2]/div/div[1]/div/div[4]/div[3]/span[1]").click()


# Click on 31st
driver.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[1]/div[2]/div/div[2]/div/div[1]/div/div[4]/div[2]/div/span[41]").click()


In [52]:
# Click on More Races to get the full list on the screen & Wait for Visibility of Races
maxIterations = 10 #-1 for ALL
i = 0
try:
    btnSeeMore = driver.find_element_by_css_selector('button.btn-itra-black[type="button"]')    

except:
    btnSeeMore = None
    print("No button")

while btnSeeMore is not None and i < maxIterations:
    i = i +1
    btnSeeMore.click()
    try:
        btnSeeMore = driver.find_element_by_css_selector('button.btn-itra-black[type="button"]')    

    except:
        btnSeeMore = None
        print("No button")      
    finally:
        time.sleep(5)

print("No more results")

No more results


In [66]:


def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    #Prepare browser
    #Setup options for FireFox
    from selenium.webdriver.firefox.options import Options as FirefoxOptions

    options = FirefoxOptions()
    options.binary_location = r'C:\Users\apelaez\AppData\Local\Mozilla Firefox\firefox.exe'

    # Using selenium, open firefox window with the ITRA website
    driver = webdriver.Firefox(options=options)
    driver.get("https://itra.run/races")

    # Getting current URL source code 
    get_title = driver.title 
    
    # Printing the title of this URL 
    print(get_title) 
    assert "- ITRA" in driver.title
    logging.info("Thread %s: finishing", name)

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(thread_function, range(3))

18:43:40: Thread 0: starting
18:43:40: Thread 1: starting
18:43:40: Thread 2: starting
18:44:00: Thread 1: finishing
Races - ITRA
18:44:01: Thread 2: finishing
Races - ITRA
18:44:05: Thread 0: finishing
Races - ITRA


In [31]:
# Getting current URL source code 
get_source = driver.page_source
time.sleep(2)

In [32]:
# Scraping race names with BeautifulSoup
soup = BeautifulSoup(get_source, 'html')
#print(soup.h5)
#soup.find_all('h5')

In [33]:
# Find the race names
racesList = re.findall(r'(?<=<h5 data-v-f3c4ac1c="" class="itra-green">)(.*?)(?=</h5>)', get_source)
print(len(racesList))
print(racesList)

2100
['New Year Ne Mountain Race 2021 2021 - Challenge - Solo 23.60 K', 'New Year Ne Mountain Race 2021 2021 - Challenge - Team 23.60 K', 'New Year Ne Mountain Race 2021 2021 - Elite - Solo 36.80 K', 'New Year Ne Mountain Race 2021 2021 - Elite - Team 36.80 K', 'Wuxi New Year Blessing Ultra Trail 2021 - Wuxi 25km 24.30 K', 'Hunting Trail 2021 - Black Bear 101.60 K', 'Hunting Trail 2021 - Sambar - 狩獵山徑-台灣水鹿 82.90 K', 'Hunting Trail 2021 -  Goat 47.90 K', 'San Tan Scramble 2021 - 50k 50.10 K', 'San Tan Scramble 2021 - 26k 26.00 K', 'Hunting Trail 2021 - Wild Boar 39.30 K', 'Country To Capital 2021 69.60 K', 'Vagamon Ultrail 2021 - 60k 60.30 K', 'Vagamon Ultrail 2021 - 90k 90.10 K', 'S1 Trail La Corsa Della Bora 2021 - S1 Night Trail 80 Km - 79.70 K', 'Hoka One One Bandera Endurance Trail Run 2021 - 100 Km 99.50 K', 'Hoka One One Bandera Endurance Trail Run 2021 - 50 Km 50.40 K', 'S1 Trail La Corsa Della Bora 2021 - S1 Challenge 120km 121.10 K', 'Montane® Spine® Races 2021 - Mrt Challenge

In [34]:
# Find the mouse over link pointing to the race site
links = [a['href'] for a in soup.find_all('a',"card ontop", href=True)]
print(len(links))
print(links)

2100
['https://itra.run/race/13893', 'https://itra.run/race/13895', 'https://itra.run/race/13897', 'https://itra.run/race/13899', 'https://itra.run/race/23623', 'https://itra.run/race/25353', 'https://itra.run/race/25355', 'https://itra.run/race/25357', 'https://itra.run/race/2642', 'https://itra.run/race/23465', 'https://itra.run/race/25359', 'https://itra.run/race/822', 'https://itra.run/race/9497', 'https://itra.run/race/9423', 'https://itra.run/race/26317', 'https://itra.run/race/931', 'https://itra.run/race/1781', 'https://itra.run/race/26747', 'https://itra.run/race/5963', 'https://itra.run/race/1628', 'https://itra.run/race/17069', 'https://itra.run/race/4050', 'https://itra.run/race/4049', 'https://itra.run/race/10747', 'https://itra.run/race/24893', 'https://itra.run/race/2740', 'https://itra.run/race/1285', 'https://itra.run/race/27887', 'https://itra.run/race/27888', 'https://itra.run/race/27906', 'https://itra.run/race/27907', 'https://itra.run/race/28044', 'https://itra.ru

In [35]:
# Scraping the data for distance, elevation gain and loss
myList = re.findall(r'(?<=<span class="icon-text-grey icon-bold">)(.*?)(?=</span>)', get_source)
#print(myList)
#len(myList)

# Find the race distance
distancesList = myList[0::3]
print(len(distancesList))

# Find the race elevation gain
gainList = myList[1::3]
print(len(gainList))

# Find the race elevation loss
lossList = myList[2::3]
print(len(lossList))

# Find the race date
datesList = re.findall(r'(?<=<span data-v-f3c4ac1c="" class="itra-grey" style="margin-top: 0.2rem; margin-left: 0.2rem; margin-right: 2rem; font-size: 80%;">)(.*?)(?=</span>)', get_source)
#print(datesList)
print(len(datesList))

# Loop
# Visit race page in Itra
# Scraping www, place, topology, number of participants

2100
2100
2100
2100


In [137]:
# Find the number of finishers of the race
#<span class="icon-finisher icon-bold">370</span>
finishersList = re.findall(r'(?<=<span class="icon-finisher icon-bold">)(.*?)(?=</span>)', get_source)
print(finishersList)
len(finishersList)

['300', '300', '380', '380', '220', '340', '300', '240', '200', '300', '280', '310', '250', '240', '230', '280', '160', '210', '220', '240', '230', '140', '350', '190', '220', '210', '220', '360', '280', '230', '310', '300', '320', '260', '330', '300', '260', '280', '290', '280', '400', '320', '390', '210', '390', '370', '270', '210', '190', '330', '290', '340', '270', '250', '250', '90', '320', '260', '360', '370', '360', '260', '170', '270', '230', '320', '330', '130', '120', '370', '310', '190', '290', '160', '380', '310', '200', '300', '240', '210', '180', '330', '240', '340', '260', '260', '210', '340', '360', '450', '390', '430', '350', '200', '280', '310', '390', '360', '270', '360', '300', '10', '260', '210', '210', '430', '490', '310', '190', '260', '300', '140', '360', '350', '310', '340', '380', '260', '240', '280', '390', '260', '370', '350', '290', '360', '310', '310', '260', '260', '330', '350', '340', '240', '330', '200', '200', '210', '250', '290', '190', '250', '310', 

2566

In [136]:
# Assign data to tuples: # get the list of tuples from two lists and merge them by using zip(). 
list_of_tuples = list(zip(racesList, links, distancesList, gainList, lossList, datesList)) 
# Converting lists of tuples into pandas Dataframe. 
df = pd.DataFrame(list_of_tuples, columns = ['Name', 'Link', 'Distance', 'Gain', 'Loss', 'Date'])
df



,Name,Link,Distance,Gain,Loss,Date
0,New Year Ne Mountain Race 2021 2021 - Challeng...,https://itra.run/race/13893,23.60 km,+1570m,-1570m,01/01/2021
1,New Year Ne Mountain Race 2021 2021 - Challeng...,https://itra.run/race/13895,23.60 km,+1570m,-1570m,01/01/2021
2,New Year Ne Mountain Race 2021 2021 - Elite - ...,https://itra.run/race/13897,36.80 km,+2430m,-2430m,01/01/2021
3,New Year Ne Mountain Race 2021 2021 - Elite - ...,https://itra.run/race/13899,36.80 km,+2430m,-2430m,01/01/2021
4,Wuxi New Year Blessing Ultra Trail 2021 - Wuxi...,https://itra.run/race/23623,24.30 km,+1470m,-1480m,01/01/2021
...,...,...,...,...,...,...
2035,K-Way Skyrun 2021 - K-Way Skyrun 38km 37.30 K,https://itra.run/race/16943,37.30 km,+1350m,-1330m,20/11/2021
2036,Best Of San Diego 100 Miler 2021 - 50 Miler 80...,https://itra.run/race/17375,80.90 km,+1230m,-1110m,20/11/2021
2037,Best Of San Diego 100 Miler 2021 - 100 Miler 1...,https://itra.run/race/17647,161.10 km,+2480m,-2480m,20/11/2021
2038,Panda Trail By Utmb® 2021 - Mpt 103.80 K,https://itra.run/race/23021,103.80 km,+6760m,-6770m,20/11/2021


In [ ]:
# CAL DESAR LES DADES EN UN FORMAT ADIENT: JSON? CSV directament ja que l'enunciat de la PRACTICA requereix CSV

# Convert results to json and save the file
with open('result.json', 'w') as fp:
    json.dump(sample, fp)

In [230]:





print(iList)
print(iiList)
print(iiList_of_tuples)

['Distance', 'Dénivelé positif', 'Dénivelé négatif', 'Date et heure de départ', 'Lieu de départ', "Lieu d'arrivée", 'Ravitallements', 'Temps maximum', 'Participation', 'Topologie', 'Nombre de participants', 'Challenge', 'Championnat', 'Distance', 'Dénivelé positif', 'Dénivelé négatif', 'Date et heure de départ', 'Lieu de départ', "Lieu d'arrivée", 'Ravitallements', 'Temps maximum', 'Participation', 'Topologie', 'Nombre de participants', 'Challenge', 'Championnat']
['23.60km (15miles)', '1570m', '1570m', ' 01/01/2021 09:15:00', 'Tai Mei Tuk', 'Tai Mei Tuk', '1', '6 heures    ', 'solo', '85% sentiers / 5% pistes / 10% routes', '400', 'N/A', 'N/A', '23.60km (15miles)', '1570m', '1570m', ' 01/01/2021 09:15:00', 'Tai Mei Tuk', 'Tai Mei Tuk', '1', '6 heures    ', 'solo', '85% sentiers / 5% pistes / 10% routes', '400', 'N/A', 'N/A']
[('Distance', '23.60km (15miles)'), ('Dénivelé positif', '1570m'), ('Dénivelé négatif', '1570m'), ('Date et heure de départ', ' 01/01/2021 09:15:00'), ('Lieu de d

In [256]:
# Ara ataquem les dades de la pàgina específica de cada cursa


# List with race websites
websitesList = []

# List with race place and country
locationList = []

# List containing the table for each website (as a list)
tableList = []


#slicedLinks = links[:5]

for i in tqdm(links):
    print(i)
    # Using selenium, open firefox window with the ITRA website
    driver2 = webdriver.Firefox()
    #driver2.get("https://itra.run/race/13893")
    driver2.get(i)
    
    
    try:
        time.sleep(3)   

        # Click dropdown menu for language selection
        driver2.find_element_by_xpath("/html/body/div[1]/div[1]/nav/div[4]").click()

        # Select language EN
        driver2.find_element_by_xpath("/html/body/div[1]/div[1]/nav/div[4]/div/div[1]").click()

        time.sleep(15)
        
        # if link is broken, go back
    except TimeoutException:
        driver2.back()
        print("Time out exception.")
        # continue so we can return to beginning of loop
        continue

    # if you reach this point, the link is valid, and you can 'do stuff' on the page
    
    # Getting current URL source code 
    get_source2 = driver2.page_source
    time.sleep(2)   
    driver2.close()

    # Scraping race names with BeautifulSoup
    soup2 = BeautifulSoup(get_source2, 'html')

    # List containing the race website (when available) and "facebook", "twitter" and other info
    hrefList = [a['href'] for a in soup2.find_all('a', {'rel': "ugc"}, href=True)]

    # Remove the links that contain "facebook" or "twitter" or "@"
    hrefList[:] = [x for x in hrefList if "facebook" not in x]
    hrefList[:] = [x for x in hrefList if "twitter" not in x]
    hrefList[:] = [x for x in hrefList if "@" not in x]

    print(len(hrefList))
    print(hrefList) 
    websitesList.append(hrefList)
    
    # Srape the table with additional data: data labels (first) and content
    labelsList  = re.findall(r'(?<=<div class="colinforace1">)(.*?)(?=</div>)', get_source2)
    contentList = re.findall(r'(?<=<div class="colinforace2 mbb">)(.*?)(?=</div>)', get_source2)

    labels_content_list_of_tuples = list(zip(labelsList[:13], contentList[:13])) 
    print(labels_content_list_of_tuples)
    tableList.append(labels_content_list_of_tuples)
    
    # Scrape the location
    location = soup2.find('p').getText()
    print(location)
    locationList.append(location)
    
    # Check scraped data is saved in a manner that can be directly matched
    print(len(websitesList))
    print(len(locationList))
    print(len(tableList))
    
print(websitesList)

  0%|          | 0/2040 [00:00<?, ?it/s]

https://itra.run/race/13893


  0%|          | 1/2040 [00:43<24:37:41, 43.48s/it]

1
['http://xterace.com/NE-Mountain-Race-2021/']
[('Distance', '23.60km (15miles)'), ('Elevation gain', '1570m'), ('Elevation loss', '1570m'), ('Start time', ' 01/01/2021 09:15:00'), ('Start location', 'Tai Mei Tuk'), ('Finish location', 'Tai Mei Tuk'), ('Aid station', '1'), ('Time limit', '6 hours    '), ('Participation', 'solo'), ('Topology', '85% Paths / 5% Tracks / 10% Roads'), ('Number of participants', '400'), ('Challenge', 'N/A'), ('Championship', 'N/A')]

                                        Hok Tau, Hong Kong, China
https://itra.run/race/13895


  0%|          | 2/2040 [01:20<22:32:39, 39.82s/it]

...

1
['http://www.translantau.com/']
[('Distance', '103.30km (64miles)'), ('Elevation gain', '5160m'), ('Elevation loss', '5170m'), ('Start time', ' 26/02/2021 23:30:00'), ('Start location', 'Mui Wo'), ('Finish location', 'Mui Wo'), ('Aid station', '9'), ('Time limit', '33 hours    '), ('Participation', 'solo'), ('Topology', '10% Paths / 70% Tracks / 20% Roads'), ('Challenge', 'N/A'), ('Championship', 'N/A'), ('Distance', '103.30km (64miles)')]

                                        Lantau Island, Hong Kong, China
https://itra.run/race/25547


 11%|█         | 218/2040 [11:46:51<64:00:33, 126.47s/it]

1
['http://fourfeathersultra.com']
[('Distance', '90.20km (56miles)'), ('Elevation gain', '2960m'), ('Elevation loss', '3610m'), ('Start time', ' 27/02/2021 06:00:00'), ('Start location', 'Kiau Nulu'), ('Finish location', 'Melangkap Baru'), ('Aid station', '9'), ('Time limit', '28 hours    '), ('Participation', 'solo'), ('Topology', '35% Paths / 55% Tracks / 10% Roads'), ('Number of participants', '200'), ('Challenge', 'N/A'), ('Championship', 'N/A')]

                                        Kota Kinabalu, Malaysia


 11%|█         | 218/2040 [12:00:09<100:18:52, 198.21s/it]


TimeoutException: Message: Timeout loading page after 300000ms
